#Land Cover Classification using Random Forests on LANDSAT-8 data
The original code that this notebook is based on can be found on Google Earth Engine at the following URL:

https://code.earthengine.google.com/?scriptPath=users%2Freetam1%2Flandsat8%3APanama_LC_LANDSAT8_SR

It is recommended that this notebook is run in Google Colab, because it was written in Colab. What this means is that a lot of the code takes advantage of Colab features like hiding cells, comments and titles, that might not work very well with regular Jupyter notebooks. It, however, _is_ entirely compatible with Jupyter notebooks.

##Current issues with the code:
1. Layering of images is tricky, maybe a modification of the folium code will help. At this point, there are 2 separate image outputs, one using classification on the median image, and another on the entire imagecollection.
2. The dictionary for the Random Forest, which contains information such as out of bounds error as well as variable importance, cannot be displayed in a user friendly way yet. The information however is available and the output has been suppressed until a way to display it properly is found.
3. Need a way to display `ee.image` objects as histograms. There should be a way to save them in a `numpy` array and display its statistics.

In [ ]:
# @title Install packages - one-time if running on your own desktop, but needs to be done every time if on Google Colab

!pip install geehydro
!pip install folium
!pip install geeadd

In [ ]:
# @title Import packages 
import time
import ee 
import folium
import geehydro
from datetime import datetime as dt
from IPython.display import Image
from tabulate import tabulate
import numpy as np
from osgeo import gdal
from osgeo import osr
import geeadd

GEE authentication details for Colab can be found at the following link:

https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb

In [ ]:
# @title GEE authentication - requires token
ee.Authenticate()


In [ ]:
# initialize the connection to the server
ee.Initialize()

In [ ]:
# @title Authenticate Google Drive so that files saved there can be imported and worked on
from google.colab import drive
drive.mount('/drive')

# Data, Parameters and Functions
## Data Imports
Data importing is one of the sections that is cumbersome if using Jupyter notebooks, since there is no easy way to hide cells as far as we know. It would be nice if the Python API can use the imports from a GEE script directly instead of having to declare them all here.

##Parameters that can be changed by the user
This includes the starting and the ending date, but also RF hyperparameters.
1. `startDate` and `endDate` must be consistent with `season`
2. `numberOfTrees` has shown okay performance with 10. Ideal number ~100 just to be sure.
3. `variablesPerSplit`, `minLeafPopulation`, `bagFraction` don't really need to be changed. Check rule of thumb for the first one.
4. `isNDVI` decides whether the classification should include NDVI or not. Any value that is not `False` is taken to be true as a safeguard.
5. `season` needs to be consistent with `startDate` and `endDate`:\
a. `annual` has `startDate` **Jan 1** and `endDate` **Dec 31**.\
b. `dry` has `startDate` **Jan 15** and `endDate` **Mar 31**. \
c. `wet` has `startDate` **May 1** and `endDate` **Dec 15**.
6. `analysisScale` is the resolution (in metres) of the analysis throughout the file. For Landsat the default resolution is 30.

## External datasets applied to the imagecollection through functions:
1. Matt Hansen's water mask
2. NDVI band
3. SRTM
4. Cloud mask based on Landsat's function


In [ ]:
# @title Data import (3 classes)
srtm = ee.Image("USGS/SRTMGL1_003")
lc2012 = ee.Image("users/reetam1/panama/Cobertura_Boscosa_2012_50k_all")
roi = ee.Geometry.Polygon(
        [[[-83.11420630932491, 9.67791124821118],
          [-83.11420630932491, 7.00411556169122],
          [-77.09369849682491, 7.00411556169122],
          [-77.09369849682491, 9.67791124821118]]], None, False)
hansenImage = ee.Image("UMD/hansen/global_forest_change_2015")
shape = ee.FeatureCollection("users/reetam1/panama/Panama_Shapefile")
protectedAreas = ee.FeatureCollection("users/reetam1/panama/Panamas_Protected_National_Parks_Layer-shp")
lc2019 = ee.Image("users/reetam1/panama/PanamaLC2019")
lc2019_3c = ee.Image("users/reetam1/panama/lc2019_3c")
l8_tr_2019a = ee.Image("users/reetam1/panama/Landsat8_training_2019a")
l8_tr_2019d = ee.Image("users/reetam1/panama/Landsat8_training_2019d")
l8_tr_2019w = ee.Image("users/reetam1/panama/Landsat8_training_2019w")
lc2019_masked = ee.Image("users/reetam1/panama/PanamaLC2019_masked")
forest = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[-77.63070575983168, 7.875953392422914],
                  [-77.63070575983168, 7.850446292051472],
                  [-77.60083668024184, 7.850446292051472],
                  [-77.60083668024184, 7.875953392422914]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-77.73850910455825, 7.863370085570584],
                  [-77.73850910455825, 7.837522096582117],
                  [-77.7141331890309, 7.837522096582117],
                  [-77.7141331890309, 7.863370085570584]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-77.43638508112075, 7.842963912045321],
                  [-77.43638508112075, 7.812012639012329],
                  [-77.40788929254653, 7.812012639012329],
                  [-77.40788929254653, 7.842963912045321]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-78.2591213179938, 7.832794461316137],
                  [-78.2591213179938, 7.8181690639533175],
                  [-78.2426418258063, 7.8181690639533175],
                  [-78.2426418258063, 7.832794461316137]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-77.67993583215396, 8.380358456232164],
                  [-77.67993583215396, 8.369149617916849],
                  [-77.66860618127505, 8.369149617916849],
                  [-77.66860618127505, 8.380358456232164]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.4831712977281, 9.375302962012668],
                  [-79.4831712977281, 9.367342555698475],
                  [-79.47510321301131, 9.367342555698475],
                  [-79.47510321301131, 9.375302962012668]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.40832693737654, 9.402570202891003],
                  [-79.40832693737654, 9.396812082340428],
                  [-79.40111715954451, 9.396812082340428],
                  [-79.40111715954451, 9.402570202891003]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.29211218517928, 9.39079982486129],
                  [-79.29211218517928, 9.386142370583755],
                  [-79.28756315869002, 9.386142370583755],
                  [-79.28756315869002, 9.39079982486129]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.61560805004744, 9.484443666826042],
                  [-79.61560805004744, 9.480126111471085],
                  [-79.61131651562361, 9.480126111471085],
                  [-79.61131651562361, 9.484443666826042]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.19176175424631, 9.449889019792812],
                  [-79.19176175424631, 9.446417699179888],
                  [-79.1878135425764, 9.446417699179888],
                  [-79.1878135425764, 9.449889019792812]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.75760402589009, 9.167361132488104],
                  [-79.75760402589009, 9.164480151583705],
                  [-79.75459995179341, 9.164480151583705],
                  [-79.75459995179341, 9.167361132488104]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.71812190919087, 9.156006542908724],
                  [-79.71812190919087, 9.153125469937164],
                  [-79.71528949647114, 9.153125469937164],
                  [-79.71528949647114, 9.156006542908724]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.69426097779439, 9.151600186560403],
                  [-79.69426097779439, 9.148634339047456],
                  [-79.69099941163228, 9.148634339047456],
                  [-79.69099941163228, 9.151600186560403]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.7345155706899, 9.176088667220721],
                  [-79.7345155706899, 9.173631421870843],
                  [-79.73236980347798, 9.173631421870843],
                  [-79.73236980347798, 9.176088667220721]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.97943583496088, 9.245640776063073],
                  [-79.97943583496088, 9.242760432957313],
                  [-79.97626009948725, 9.242760432957313],
                  [-79.97626009948725, 9.245640776063073]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.9584931469726, 9.251147248743539],
                  [-79.9584931469726, 9.24826695070526],
                  [-79.95626154907221, 9.24826695070526],
                  [-79.95626154907221, 9.251147248743539]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.98767558105463, 9.269953320592],
                  [-79.98767558105463, 9.267835570008728],
                  [-79.9848431683349, 9.267835570008728],
                  [-79.9848431683349, 9.269953320592]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.98578730590815, 9.230899948444858],
                  [-79.98578730590815, 9.227934764949962],
                  [-79.98278323181147, 9.227934764949962],
                  [-79.98278323181147, 9.230899948444858]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.00900344715929, 8.670028181924883],
                  [-81.00900344715929, 8.664597749689923],
                  [-81.00213699208116, 8.664597749689923],
                  [-81.00213699208116, 8.670028181924883]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.87957076893663, 8.64474206454009],
                  [-80.87957076893663, 8.636935269198766],
                  [-80.8720176683507, 8.636935269198766],
                  [-80.8720176683507, 8.64474206454009]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.81365280018663, 8.63405010828353],
                  [-80.81365280018663, 8.626073372144736],
                  [-80.80455474720812, 8.626073372144736],
                  [-80.80455474720812, 8.63405010828353]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.0988242257037, 8.620569992121778],
                  [-81.0988242257037, 8.615308571794326],
                  [-81.09273024682186, 8.615308571794326],
                  [-81.09273024682186, 8.620569992121778]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.79721518639705, 8.86726454794561],
                  [-80.79721518639705, 8.863363504203168],
                  [-80.79386778954647, 8.863363504203168],
                  [-80.79386778954647, 8.86726454794561]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.76382704857967, 8.851660124530301],
                  [-80.76382704857967, 8.846147534084254],
                  [-80.75850554589412, 8.846147534084254],
                  [-80.75850554589412, 8.851660124530301]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.73150450710786, 8.96960435095491],
                  [-80.73150450710786, 8.963754409214031],
                  [-80.72472388271821, 8.963754409214031],
                  [-80.72472388271821, 8.96960435095491]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.71834090435813, 9.244348829234752],
                  [-82.71834090435813, 9.235538261459826],
                  [-82.70889952862571, 9.235538261459826],
                  [-82.70889952862571, 9.244348829234752]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.62427046978782, 9.227744114012475],
                  [-82.62427046978782, 9.217747020583923],
                  [-82.612597496155, 9.217747020583923],
                  [-82.612597496155, 9.227744114012475]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.83009246075461, 9.195887983155057],
                  [-82.83009246075461, 9.18504268810746],
                  [-82.81738951886008, 9.18504268810746],
                  [-82.81738951886008, 9.195887983155057]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.33349615039883, 8.84325919231455],
                  [-82.33349615039883, 8.837661664429245],
                  [-82.32731634082852, 8.837661664429245],
                  [-82.32731634082852, 8.84325919231455]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.2552185625082, 8.799833719663091],
                  [-82.2552185625082, 8.793387316143049],
                  [-82.24800878467617, 8.793387316143049],
                  [-82.24800878467617, 8.799833719663091]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.82228270564012, 7.503872377254315],
                  [-81.82228270564012, 7.491958824919743],
                  [-81.80820647272996, 7.491958824919743],
                  [-81.80820647272996, 7.503872377254315]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.76477614436082, 7.413491459013438],
                  [-81.76477614436082, 7.4031075246251845],
                  [-81.75293150935106, 7.4031075246251845],
                  [-81.75293150935106, 7.413491459013438]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.63603011164598, 7.356972667755864],
                  [-81.63603011164598, 7.346757658688296],
                  [-81.62658873591356, 7.346757658688296],
                  [-81.62658873591356, 7.356972667755864]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.076083773269, 7.555442280832463],
                  [-81.076083773269, 7.551443243145244],
                  [-81.07196390022213, 7.551443243145244],
                  [-81.07196390022213, 7.555442280832463]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.14840749575585, 9.042012902562872],
                  [-79.14840749575585, 9.036418428372722],
                  [-79.14239934756249, 9.036418428372722],
                  [-79.14239934756249, 9.042012902562872]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-78.80585467463195, 8.828497930370062],
                  [-78.80585467463195, 8.820186086243218],
                  [-78.79692828303038, 8.820186086243218],
                  [-78.79692828303038, 8.828497930370062]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-78.85383402949034, 8.875057752388521],
                  [-78.85383402949034, 8.86216746277782],
                  [-78.84130274897277, 8.86216746277782],
                  [-78.84130274897277, 8.875057752388521]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-78.74306304422093, 8.964146068772965],
                  [-78.74306304422093, 8.956685144857472],
                  [-78.735509943635, 8.956685144857472],
                  [-78.735509943635, 8.964146068772965]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-78.88257650441753, 8.336282987054265],
                  [-78.88257650441753, 8.317089733835344],
                  [-78.86249212331401, 8.317089733835344],
                  [-78.86249212331401, 8.336282987054265]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.97132732850362, 8.48539945841446],
                  [-79.97132732850362, 8.483298396888244],
                  [-79.96935322266866, 8.483298396888244],
                  [-79.96935322266866, 8.48539945841446]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.0941746623497, 8.243827670738781],
                  [-82.0941746623497, 8.241279348944012],
                  [-82.09138516497421, 8.241279348944012],
                  [-82.09138516497421, 8.243827670738781]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.0043571185037, 8.25652872240658],
                  [-82.0043571185037, 8.252281646772047],
                  [-82.00040890683378, 8.252281646772047],
                  [-82.00040890683378, 8.25652872240658]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.42982106085019, 8.452234543733773],
                  [-81.42982106085019, 8.4511733115319],
                  [-81.42913441534238, 8.4511733115319],
                  [-81.42913441534238, 8.452234543733773]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.9375455272643, 8.543199529447149],
                  [-79.9375455272643, 8.54014389758119],
                  [-79.93479894523306, 8.54014389758119],
                  [-79.93479894523306, 8.543199529447149]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.14826242712735, 9.447579545465539],
                  [-79.14826242712735, 9.444870206886309],
                  [-79.14491503027676, 9.444870206886309],
                  [-79.14491503027676, 9.447579545465539]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.13246958044766, 9.469253486133363],
                  [-79.13246958044766, 9.46578236095737],
                  [-79.12912218359708, 9.46578236095737],
                  [-79.12912218359708, 9.469253486133363]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.01456625223047, 8.565994405963252],
                  [-80.01456625223047, 8.563787695231763],
                  [-80.01267797708398, 8.563787695231763],
                  [-80.01267797708398, 8.565994405963252]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.73833342757577, 7.30925231425128],
                  [-80.73833342757577, 7.296992962346657],
                  [-80.72597380843514, 7.296992962346657],
                  [-80.72597380843514, 7.30925231425128]]], None, False),
            {
              "landcover": 1,
              "forestcover": 1,
              "system:index": "47"
            })])
pasture = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.0170494465119, 7.982233088126669],
                  [-81.0170494465119, 7.97390309241019],
                  [-81.00692142527167, 7.97390309241019],
                  [-81.00692142527167, 7.982233088126669]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.01687778513495, 8.014361479325585],
                  [-81.01687778513495, 8.00654210404335],
                  [-81.00743640940253, 8.00654210404335],
                  [-81.00743640940253, 8.014361479325585]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.64111114687374, 8.012577164603332],
                  [-80.64111114687374, 8.006032669029558],
                  [-80.63407303041866, 8.006032669029558],
                  [-80.63407303041866, 8.012577164603332]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.75337768740108, 8.058130733049707],
                  [-80.75337768740108, 8.052946723009354],
                  [-80.74702621645382, 8.052946723009354],
                  [-80.74702621645382, 8.058130733049707]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.57776809877804, 8.077931340137468],
                  [-80.57776809877804, 8.069008439846886],
                  [-80.56918502993038, 8.069008439846886],
                  [-80.56918502993038, 8.077931340137468]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.54082132643816, 8.559014112672902],
                  [-82.54082132643816, 8.550526538051697],
                  [-82.5318949348366, 8.550526538051697],
                  [-82.5318949348366, 8.559014112672902]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.601846945945, 8.56147547392614],
                  [-82.601846945945, 8.552478696961122],
                  [-82.59498049086687, 8.552478696961122],
                  [-82.59498049086687, 8.56147547392614]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.70830095425599, 8.71651082942035],
                  [-82.70830095425599, 8.712014315970606],
                  [-82.70409525052064, 8.712014315970606],
                  [-82.70409525052064, 8.71651082942035]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.80486047879212, 8.737168643638013],
                  [-82.80486047879212, 8.7337752413622],
                  [-82.80185640469544, 8.7337752413622],
                  [-82.80185640469544, 8.737168643638013]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.81671551449142, 7.358266239602661],
                  [-80.81671551449142, 7.352648030065465],
                  [-80.81156567318283, 7.352648030065465],
                  [-80.81156567318283, 7.358266239602661]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.57201603959571, 9.45150293088703],
                  [-82.57201603959571, 9.443713613770667],
                  [-82.56446293900977, 9.443713613770667],
                  [-82.56446293900977, 9.45150293088703]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.85652112850461, 8.690257590720597],
                  [-82.85652112850461, 8.6870122432913],
                  [-82.85338830837522, 8.6870122432913],
                  [-82.85338830837522, 8.690257590720597]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.75013955460717, 8.679144254560141],
                  [-82.75013955460717, 8.674944263430996],
                  [-82.7465775810354, 8.674944263430996],
                  [-82.7465775810354, 8.679144254560141]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.74908484565745, 7.996098437730571],
                  [-80.74908484565745, 7.98921368717271],
                  [-80.74204672920237, 7.98921368717271],
                  [-80.74204672920237, 7.996098437730571]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.81808945628568, 7.45293106635877],
                  [-80.81808945628568, 7.44595235405462],
                  [-80.80899140330716, 7.44595235405462],
                  [-80.80899140330716, 7.45293106635877]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.43719797324695, 8.142255250410681],
                  [-81.43719797324695, 8.139238964865795],
                  [-81.43136148643055, 8.139238964865795],
                  [-81.43136148643055, 8.142255250410681]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.34362063305315, 8.119969808405077],
                  [-81.34362063305315, 8.11567879079364],
                  [-81.33817038433489, 8.11567879079364],
                  [-81.33817038433489, 8.119969808405077]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.33284888164934, 8.110622978566294],
                  [-81.33284888164934, 8.106416834229742],
                  [-81.32615408794817, 8.106416834229742],
                  [-81.32615408794817, 8.110622978566294]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.3346264445444, 8.173102969183242],
                  [-81.3346264445444, 8.16932227964927],
                  [-81.33063531753024, 8.16932227964927],
                  [-81.33063531753024, 8.173102969183242]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.32694459792575, 8.177775682018865],
                  [-81.32694459792575, 8.174929581627449],
                  [-81.32321096297702, 8.174929581627449],
                  [-81.32321096297702, 8.177775682018865]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.3164732539316, 8.17522693634992],
                  [-81.3164732539316, 8.17267817439282],
                  [-81.31381250258883, 8.17267817439282],
                  [-81.31381250258883, 8.17522693634992]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.2520517389525, 8.335211821301936],
                  [-81.2520517389525, 8.331984694120333],
                  [-81.24818935797106, 8.331984694120333],
                  [-81.24818935797106, 8.335211821301936]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.24149456426989, 8.337249993169982],
                  [-81.24149456426989, 8.33249424228746],
                  [-81.23660221502672, 8.33249424228746],
                  [-81.23660221502672, 8.337249993169982]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.5591150858786, 8.208830188426113],
                  [-80.5591150858786, 8.200674781109935],
                  [-80.5510470011618, 8.200674781109935],
                  [-80.5510470011618, 8.208830188426113]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-77.99877601365314, 8.493217478175358],
                  [-77.99877601365314, 8.478955791776553],
                  [-77.98641639451252, 8.478955791776553],
                  [-77.98641639451252, 8.493217478175358]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-77.97817664841877, 8.545166275127317],
                  [-77.97817664841877, 8.529887949891474],
                  [-77.96822028855549, 8.529887949891474],
                  [-77.96822028855549, 8.545166275127317]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-77.9086663180465, 8.313038683672794],
                  [-77.9086663180465, 8.304205983004879],
                  [-77.90094155608361, 8.304205983004879],
                  [-77.90094155608361, 8.313038683672794]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-78.3376161749622, 8.650880346918115],
                  [-78.3376161749622, 8.640358277962987],
                  [-78.33109304263799, 8.640358277962987],
                  [-78.33109304263799, 8.650880346918115]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-78.23795381907887, 8.58083093515109],
                  [-78.23795381907887, 8.572004359857587],
                  [-78.23108736400074, 8.572004359857587],
                  [-78.23108736400074, 8.58083093515109]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.78850116017755, 8.760895201510332],
                  [-82.78850116017755, 8.758010995330551],
                  [-82.78506793263848, 8.758010995330551],
                  [-82.78506793263848, 8.760895201510332]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.3393885342687, 8.242989230694063],
                  [-81.3393885342687, 8.234494742880152],
                  [-81.32994715853629, 8.234494742880152],
                  [-81.32994715853629, 8.242989230694063]]], None, False),
            {
              "landcover": 3,
              "forestcover": 0,
              "system:index": "30"
            })])
nonforest = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.4382113530409, 8.430531407257824],
                  [-82.4382113530409, 8.411512615818868],
                  [-82.42001524708387, 8.411512615818868],
                  [-82.42001524708387, 8.430531407257824]]], None, False),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.07518051663534, 8.251657682218632],
                  [-82.07518051663534, 8.243503156154427],
                  [-82.0672840932955, 8.243503156154427],
                  [-82.0672840932955, 8.251657682218632]]], None, False),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.04668472806112, 8.264228913430161],
                  [-82.04668472806112, 8.258283110724205],
                  [-82.03913162747519, 8.258283110724205],
                  [-82.03913162747519, 8.264228913430161]]], None, False),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-82.04599808255331, 8.27730936383766],
                  [-82.04599808255331, 8.270344502716863],
                  [-82.04084824124472, 8.270344502716863],
                  [-82.04084824124472, 8.27730936383766]]], None, False),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.8422123280438, 8.206621564524191],
                  [-81.8422123280438, 8.197616575947437],
                  [-81.83534587296568, 8.197616575947437],
                  [-81.83534587296568, 8.206621564524191]]], None, False),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-81.82779277237974, 8.182494532805212],
                  [-81.82779277237974, 8.17459346171618],
                  [-81.82109797867857, 8.17459346171618],
                  [-81.82109797867857, 8.182494532805212]]], None, False),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.39572556503553, 7.405256725224219],
                  [-80.39572556503553, 7.399298648690917],
                  [-80.3885157872035, 7.399298648690917],
                  [-80.3885157872035, 7.405256725224219]]], None, False),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.3991587925746, 7.426194469307545],
                  [-80.3991587925746, 7.419555780242993],
                  [-80.39435227401991, 7.419555780242993],
                  [-80.39435227401991, 7.426194469307545]]], None, False),
            {
              "landcover": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.39177735336561, 7.445599292935431],
                  [-80.39177735336561, 7.438790680583342],
                  [-80.38422425277967, 7.438790680583342],
                  [-80.38422425277967, 7.445599292935431]]], None, False),
            {
              "landcover": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.88521323480299, 7.373144788494047],
                  [-80.88521323480299, 7.3644623623513175],
                  [-80.87439856805494, 7.3644623623513175],
                  [-80.87439856805494, 7.373144788494047]]], None, False),
            {
              "landcover": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.64125400056976, 8.113989902947225],
                  [-80.64125400056976, 8.101753826999957],
                  [-80.62872272005218, 8.101753826999957],
                  [-80.62872272005218, 8.113989902947225]]], None, False),
            {
              "landcover": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.69103579988617, 8.07982992763996],
                  [-80.69103579988617, 8.064873405778425],
                  [-80.68125110139984, 8.064873405778425],
                  [-80.68125110139984, 8.07982992763996]]], None, False),
            {
              "landcover": 2,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.6355891751303, 8.153244795568254],
                  [-80.6355891751303, 8.144408531750694],
                  [-80.6242595242514, 8.144408531750694],
                  [-80.6242595242514, 8.153244795568254]]], None, False),
            {
              "landcover": 2,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.58271747102874, 8.16225078663162],
                  [-80.58271747102874, 8.145258180980486],
                  [-80.57585101595063, 8.145258180980486],
                  [-80.57585101595063, 8.16225078663162]]], None, False),
            {
              "landcover": 2,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.50924640169282, 8.139480530665404],
                  [-80.50924640169282, 8.126395544050212],
                  [-80.49482684602876, 8.126395544050212],
                  [-80.49482684602876, 8.139480530665404]]], None, False),
            {
              "landcover": 2,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.49242358675141, 8.070312205064209],
                  [-80.49242358675141, 8.05178599528831],
                  [-80.47748904695649, 8.05178599528831],
                  [-80.47748904695649, 8.070312205064209]]], None, False),
            {
              "landcover": 2,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.48352501387262, 8.319277918235139],
                  [-80.48352501387262, 8.298555059224782],
                  [-80.46155235762262, 8.298555059224782],
                  [-80.46155235762262, 8.319277918235139]]], None, False),
            {
              "landcover": 2,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.2627684831109, 8.341358447096836],
                  [-80.2627684831109, 8.32539265094904],
                  [-80.24388573164606, 8.32539265094904],
                  [-80.24388573164606, 8.341358447096836]]], None, False),
            {
              "landcover": 2,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-80.42378685469293, 8.395705196336303],
                  [-80.42378685469293, 8.384836453530482],
                  [-80.41245720381403, 8.384836453530482],
                  [-80.41245720381403, 8.395705196336303]]], None, False),
            {
              "landcover": 2,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.7903563737359, 8.88583469968475],
                  [-79.7903563737359, 8.869552631498907],
                  [-79.77181694502497, 8.869552631498907],
                  [-79.77181694502497, 8.88583469968475]]], None, False),
            {
              "landcover": 2,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.64822075361872, 8.950955726006605],
                  [-79.64822075361872, 8.940103028835422],
                  [-79.636204457232, 8.940103028835422],
                  [-79.636204457232, 8.950955726006605]]], None, False),
            {
              "landcover": 2,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.5246583112779, 9.082536967834034],
                  [-79.5246583112779, 9.064229519877655],
                  [-79.50062571850447, 9.064229519877655],
                  [-79.50062571850447, 9.082536967834034]]], None, False),
            {
              "landcover": 2,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.51779185619978, 9.023542963513643],
                  [-79.51779185619978, 9.005910696187897],
                  [-79.49444590893415, 9.005910696187897],
                  [-79.49444590893415, 9.023542963513643]]], None, False),
            {
              "landcover": 2,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.34845009178638, 9.079274169005375],
                  [-79.34845009178638, 9.06317029833776],
                  [-79.33454552025317, 9.06317029833776],
                  [-79.33454552025317, 9.079274169005375]]], None, False),
            {
              "landcover": 2,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.40527000755786, 9.094868807025886],
                  [-79.40527000755786, 9.079952210858417],
                  [-79.393768695302, 9.079952210858417],
                  [-79.393768695302, 9.094868807025886]]], None, False),
            {
              "landcover": 2,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[-79.3139461550188, 9.07028999343499],
                  [-79.3139461550188, 9.059440877909688],
                  [-79.30707969994067, 9.059440877909688],
                  [-79.30707969994067, 9.07028999343499]]], None, False),
            {
              "landcover": 2,
              "system:index": "25"
            })])

In [ ]:
lc2019_reprojected = lc2019.reproject('EPSG:4326', None, 20)
lc2019_reprojected_3c = lc2019_3c.reproject('EPSG:4326', None, 20)
lc2019 = lc2019_reprojected
lc2019_3c = lc2019_reprojected_3c

In [ ]:
#@title Function definitions


#Function to mask clouds based on the pixel_qa band of Landsat 8 SR data.
def maskL8sr(image): 
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

# Function to apply M. Hansen's water mask to an image
def maskwater(image):
    # Select the land/water mask.
    mask = lc2019.lte(3)
    # Update the composite mask with the water mask.
    return image.updateMask(mask)

def maskwater2(image):
    # Select the land/water mask.
    datamask = lc2019_masked.select('mask')
    mask = datamask.eq(1)
    # Update the composite mask with the water mask.
    return image.updateMask(mask)

#Function for computing NDVI for each image in collection
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


#Function for adding elevation to each image in collection
def addDEM(image):
    elevation = srtm.select('elevation')
    slope = ee.Terrain.slope(elevation)
    image2 = image.addBands(elevation)
    return image2.addBands(slope)


#function for running RF on each image in a collection
def RFcollection(image):
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI', 'elevation', 'slope']
    LCclassified = image.select(bands).classify(classifier).rename('LCclassified') 
    return image.addBands(LCclassified)


#Function for identifying pixels with data
def isCloudy(image):
    original = image.select('NDVI')
    cloudStatus1 = image.where(original.gte(-1), 0)
    cloudStatus  = cloudStatus1.where(original.gte(-1).And(original.lte(1)), 1).select('NDVI').rename('cloudstatus')
    return image.addBands(cloudStatus)

def pasture2nf(image):
  class1 = image.select('LCclassified')
  class2 = class1.where(class1.gt(1),0)
  mask = class2.gte(0)
  return class2.updateMask(mask)
  
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


In [ ]:
# @title Specify parameters for the analysis
year = '2020'
season = 'w'
numberOfTrees = 300 #more trees = more generalization, less overfitting, better results, but also longer runtime
variablesPerSplit = 4
minLeafPopulation = 5
bagFraction = 0.7
analysisScale = 20
#lc2019 = maskwater(lc2019)

In [ ]:
if season == 'a':
  startDate = year + '-01-01'
  endDate = year + '-12-31'
  cutoff = 0.45
elif season == 'w':
  startDate = year + '-05-01'
  endDate = year + '-12-15'
  cutoff = 0.5
else:
  startDate = year + '-01-15'
  endDate = year + '-03-31'
  cutoff = 0.45
compositeFile = 'L8_Me_'+ year + '_' + str(analysisScale) + season
meClsFile = 'L8_Cls_Me_'+ year + '_' + str(analysisScale) + season #+ '_3c'
meAccFile = 'L8_Acc_Me_'+ year + '_' + str(analysisScale) + season #+ '_3c'
icClsFile = 'L8_Cls_IC_'+ year + '_' + str(analysisScale) + season #+ '_3c'
icAccFile = 'L8_Acc_IC_'+ year + '_' + str(analysisScale) + season #+ '_3c'
totPixFile = 'L8_Pix_'+ year + '_' + str(analysisScale) + season

In [ ]:
# @title Load input datasets
# Load the Landsat 8 scaled radiance image collection.
landsatCollection = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterDate(startDate, endDate).filterBounds(shape).map(maskL8sr).map(maskwater).map(addDEM).map(addNDVI))

count = landsatCollection.size()
#print('Number of Images in Collection: ', str(count.getInfo()))

# Load the 2014 median composite on which the classifier will be trained
if season=='dry':
    composite_training = maskwater(l8_tr_2019d)
elif season=='wet': 
    composite_training = maskwater(l8_tr_2019w)
else:
    composite_training = maskwater(l8_tr_2019a)

In [ ]:
count = landsatCollection.size()
print('Number of Images in Collection: ', str(count.getInfo()))

Number of Images in Collection:  216


# Land cover classification on Median layer based on Landsat-8 SR
##Bands used:
1. B2-B7 from the Landsat-8 SR data
2. NDVI computed from the same data
3. Elevation and slope bands based on SRTM

In [ ]:
# @title Add median composite and original data
composite = maskwater(landsatCollection.median())
#print(composite.bandNames().getInfo())
#composite_training = composite

In [ ]:
#@title Prepare data for classification
# Merge the three geometry layers into a single FeatureCollection.
newfc = forest.merge(nonforest).merge(pasture)
#print(newfc, 'newfc')

# Select the bands for training
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'elevation', 'slope', 'NDVI']

# Sample the input imagery to get a FeatureCollection of training data.
training = composite_training.select(bands).sampleRegions(
  collection= newfc, 
  properties= ['landcover'], 
  scale= analysisScale)

In [ ]:
bands = ['B2', 'B3', 'B4','B5', 'B6', 'B7', 'NDVI']

composite2 = composite.select(bands).clip(newfc).toFloat()
compositeExport = ee.batch.Export.image.toDrive(image=composite2.unmask(-9999),
                                     region=roi.getInfo()['coordinates'],
                                     description=compositeFile,
                                     folder='GEE_exports',
                                     fileNamePrefix=compositeFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
compositeExport.start()
compositeExport.status()

{'creation_timestamp_ms': 1648593111332,
 'description': 'L8_Me_2019_20a',
 'id': '4WWBHQAZUYCWMCXJMMXI74MP',
 'name': 'projects/earthengine-legacy/operations/4WWBHQAZUYCWMCXJMMXI74MP',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1648593111332}

In [ ]:
compositeExport = ee.batch.Export.table.toDrive(collection =  newfc,
                                        folder = 'GEE_exports',
                                        description = compositeFile,
                                        fileNamePrefix=icClsFile,
                                        fileFormat = 'SHP')

compositeExport.start()
compositeExport.status()

{'creation_timestamp_ms': 1648401930640,
 'description': 'L8_Me_2019_20w',
 'id': 'X7L4GPHTN7AIPEJUSQLU6ENO',
 'name': 'projects/earthengine-legacy/operations/X7L4GPHTN7AIPEJUSQLU6ENO',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1648401930640}

In [ ]:
#@title Random Forest classification on median
# Make a Random Forest classifier and train it.
classifier = ee.Classifier.smileRandomForest(numberOfTrees, variablesPerSplit, minLeafPopulation, bagFraction).train(
  features= training, 
  classProperty= 'landcover', 
  inputProperties= bands
)

# Classify the input imagery.
classified = composite.select(bands).classify(classifier).rename('LCclassified')


In [ ]:
# @title Model performance over training data
# Note - there does not seem to be a good way to display the information from dict into python
# Classifier properties and performance
dict = classifier.explain()
#print(type(dict))
# Variable importance chart
variable_importance = ee.Feature(None, ee.Dictionary(dict).get('importance'))

# Get a confusion matrix representing resubstitution accuracy.
print('Accuracy metrics for training data')
print('RF error matrix:\n', tabulate(classifier.confusionMatrix().getInfo()))
print('RF accuracy: ', classifier.confusionMatrix().accuracy().getInfo())
a = classifier.confusionMatrix().getInfo()
# print(type(a))

Accuracy metrics for training data
RF error matrix:
 -  ------  ------  -----
0       0       0      0
0  128313      58     79
0      97  137288    137
0     175     457  46709
-  ------  ------  -----
RF accuracy:  0.9967987284281214


In [ ]:
forest.area()

In [ ]:
# @title Export median classification as `.tiff` to Google Drive
classified_2c = pasture2nf(classified)
medianExport = ee.batch.Export.image.toDrive(image=classified_2c.unmask(-9999),
                                     region=roi.getInfo()['coordinates'],
                                     description=meClsFile,
                                     folder='GEE_exports',
                                     fileNamePrefix=meClsFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
medianExport.start()
medianExport.status()

{'creation_timestamp_ms': 1643464638130,
 'description': 'L8_Cls_Me_2019_20w',
 'id': 'T33WYAY7T64BDHYDAYZEPZKQ',
 'name': 'projects/earthengine-legacy/operations/T33WYAY7T64BDHYDAYZEPZKQ',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1643464638130}

In [ ]:
medianExport.status()

In [ ]:
# @title Model performance export over entire Panama based on Median Composite
classified_2c = pasture2nf(classified)
classified2 = classified.where(classified_2c.gte(0),-1)
acc1 = classified2.where(lc2019.eq(1).And(classified_2c.eq(1)), 11)
acc2 = acc1.where(lc2019.gt(1).And(classified_2c.eq(0)), 0)
acc3 = acc2.where(lc2019.gt(1).And(classified_2c.eq(1)), 1)
acc4 = acc3.where(lc2019.eq(1).And(classified_2c.eq(0)), 10)
accuracy = acc4

MedianAccExport = ee.batch.Export.image.toDrive(image=accuracy.unmask(-9999),
                                     region=roi.getInfo()['coordinates'],
                                     description=meAccFile,
                                     folder='GEE_exports',
                                     fileNamePrefix=meAccFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
MedianAccExport.start()
MedianAccExport.status()

{'creation_timestamp_ms': 1643464639007,
 'description': 'L8_Acc_Me_2019_20w',
 'id': 'L4UBH56A4YDOM72BKVCANNZX',
 'name': 'projects/earthengine-legacy/operations/L4UBH56A4YDOM72BKVCANNZX',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1643464639007}

In [ ]:
MedianAccExport.status()

In [ ]:
# @title Model performance over entire Panama based on Median composite

start = time.time()

tp = classified.eq(1).And(PanamaLC.eq(1)).rename('TruePositive')
tn = classified.eq(0).And(PanamaLC.eq(0)).rename('TrueNegative')
fp = classified.eq(1).And(PanamaLC.eq(0)).rename('FalsePositive')
fn = classified.eq(0).And(PanamaLC.eq(1)).rename('FalseNegative')

landsat8_tp = tp.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': shape,
  'scale': analysisScale,
  'maxPixels': 5e12
  #'tileScale': 4
})
landsat8_tn = tn.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': shape,
  'scale': analysisScale,
  'maxPixels': 5e12
  #'tileScale': 4
})
landsat8_fp = fp.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': shape,
  'scale': analysisScale,
  'maxPixels': 5e12
  #'tileScale': 4
})
landsat8_fn = fn.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': shape,
  'scale': analysisScale,
  'maxPixels': 5e12
  #'tileScale': 4
})
time0 = time.time() - start
print(landsat8_tp.getInfo())
time1 = time.time() - start
print(landsat8_tn.getInfo())
time2 = time.time() - start
print(landsat8_fp.getInfo())
time3 = time.time() - start
print(landsat8_fn.getInfo())
time4 = time.time() - start

print('\nTime taken for each section:')
print(time1-time0)
print(time2-time1)
print(time3-time2)
print(time4-time3)

In [ ]:
#@title Display the classification result for median data

# Define a palette for the Land Use classification.

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

visParams2 = {
  'bands': ['b1'],
  'min': 1,
  'max': 0,
  'gamma': 0.4,
  'opacity': 0.6
}

visParams3 = {
  'min': 0,
  'max': 1,
  'opacity': 0.6,
  'palette': [
  '0000FF', # urban (0)  // grey
  '008000' #  forest (1) // green
  #'0000FF' // water (2)  // blue
]
}

# Create a folium map object.
Panama_map = folium.Map(location=[8.384, -78.9165], zoom_start=7.5, height=1000)

# Add the elevation model to the map object.
Panama_map.add_ee_layer(composite, visParams, 'Median')

#Panama_map.add_ee_layer(PanamaLC, visParams2, 'Original 2012 Data')

Panama_map.add_ee_layer(classified, visParams3, 'Land Use Classification on median data')

# Add a layer control panel to the map.
Panama_map.add_child(folium.LayerControl())

# Display the map.
display(Panama_map)

In [ ]:
!cat /drive/My\ Drive/GEE_exports/test.txt

# Classifying on the imagecollection and interpreting it
To classify all images of the imagecollection, `map()` is used to apply the RF classifier over each image in the collection.

There are various ways to interpret it, but the most straightforward way is to look at accuracy; what proportion of images in the imagecollection have correctly classified this pixel?

Accuracy will range from 0-1. Of course, it lacks a bit of context, since if a pixel somehow has only 1 image with information, its accuracy will be either 0 or 1. So it is usually a good idea to accompany this accuracy map with a map of how many images each pixel is based on. This information is available in the variable `totalPixels`; we have not included it in the map but it should be an added deliverable.

In [ ]:
#@title Random Forest classification on imagecollection

LCcollection = landsatCollection.map(RFcollection)
# print(LCcollection)
LCmode = LCcollection.select(['LCclassified']).mode().rename('ClassificationOnCollection')
# print(LCmean.getInfo())

In [ ]:
#@title Random Forest classification on imagecollection with NF set to 0

LCcollection = landsatCollection.map(RFcollection).map(pasture2nf)
# print(LCcollection)
LCmean = LCcollection.select(['LCclassified']).mean().rename('ClassificationOnCollection')
# print(LCmean.getInfo())

In [ ]:
# @title Export IC classification as `.tiff` to GDrive as percentage

RFmeanExport = ee.batch.Export.image.toDrive(image=LCmean.unmask(-9999),
                                     region=roi.getInfo()['coordinates'],
                                     description=icClsFile,
                                     folder='GEE_exports/percentages',
                                     fileNamePrefix=icClsFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
RFmeanExport.start()
RFmeanExport.status()

In [ ]:
# @title Export IC classification as `.tiff` to GDrive as 0-1

LCmean2 = LCmean.where(LCmean.gte(0),-1)
LCmean3 = LCmean2.where(LCmean.lte(cutoff).And(LCmean.gte(0)),0)
LCmean4 = LCmean3.where(LCmean.gt(cutoff),1)


RFMeanAccExport = ee.batch.Export.image.toDrive(image=LCmean4.unmask(-9999),
                                     region=roi.getInfo()['coordinates'],
                                     description = icClsFile,
                                     folder='GEE_exports',
                                     fileNamePrefix=icClsFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
RFMeanAccExport.start()
RFMeanAccExport.status()

{'creation_timestamp_ms': 1650484562523,
 'description': 'L8_Cls_IC_2020_20w',
 'id': 'PC7LKZ4IDLYMM6RIKYQV6XJE',
 'name': 'projects/earthengine-legacy/operations/PC7LKZ4IDLYMM6RIKYQV6XJE',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1650484562523}

In [ ]:
# @title Model performance export over entire Panama based on ImageCollection

LCmean2 = LCmean.where(LCmean.gte(0),-1)
acc1 = LCmean2.where(lc2019.eq(1).And(LCmean.gt(cutoff)),11)
acc2 = acc1.where(lc2019.gt(1).And(LCmean.lte(cutoff)),0)
acc3 = acc2.where(lc2019.gt(1).And(LCmean.gt(cutoff)),1)
acc4 = acc3.where(lc2019.eq(1).And(LCmean.lte(cutoff)),10)
accuracy=acc4;

RFMeanAccExport = ee.batch.Export.image.toDrive(image=accuracy.unmask(-9999),
                                     region=roi.getInfo()['coordinates'],
                                     description = icAccFile,
                                     folder='GEE_exports',
                                     fileNamePrefix=icAccFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
RFMeanAccExport.start()
RFMeanAccExport.status()

{'creation_timestamp_ms': 1631042724061,
 'description': 'L8_Acc_IC_2019_20d',
 'id': 'L3KWU4SHZ44T2S3L4R7U7ZXG',
 'name': 'projects/earthengine-legacy/operations/L3KWU4SHZ44T2S3L4R7U7ZXG',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1631042724061}

In [ ]:
# @title Export median classification as `.tiff` to Google Drive

RFmeanExport = ee.batch.Export.image.toDrive(image=LCmode.unmask(-9999),
                                     region=roi.getInfo()['coordinates'],
                                     description=icClsFile,
                                     folder='GEE_exports',
                                     fileNamePrefix=icClsFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
RFmeanExport.start()
RFmeanExport.status()

In [ ]:
RFmeanExport.status()

In [ ]:
# @title Model performance export over entire Panama based on ImageCollection

LCmode2 = LCmode.where(LCmode.gte(0),-1)
acc1 = LCmode2.where(lc2019.eq(1).And(LCmode.eq(1)),11)
acc2 = acc1.where(lc2019.gt(1).And(LCmode.gt(1)),0)
acc3 = acc2.where(lc2019.gt(1).And(LCmode.eq(1)),1)
acc4 = acc3.where(lc2019.eq(1).And(LCmode.gt(1)),10)
accuracy=acc4;

RFMeanAccExport = ee.batch.Export.image.toDrive(image=accuracy.unmask(-9999),
                                     region=roi.getInfo()['coordinates'],
                                     description = icAccFile,
                                     folder='GEE_exports',
                                     fileNamePrefix=icAccFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
RFMeanAccExport.start()
RFMeanAccExport.status()

In [ ]:
RFMeanAccExport.status()

In [ ]:
#@title Calculate and export the total number of visible pixels at each location over the duration
cloud = landsatCollection.map(isCloudy)
totalPixels = cloud.select('cloudstatus').sum().clip(roi).rename('TotalPixels')

totPixExport = ee.batch.Export.image.toDrive(image=totalPixels.unmask(0),
                                     region=roi.getInfo()['coordinates'],
                                     description=totPixFile,
                                     folder='GEE_exports',
                                     fileNamePrefix=totPixFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
totPixExport.start()
totPixExport.status()

{'creation_timestamp_ms': 1648081763550,
 'description': 'L8_Pix_2019_20w',
 'id': 'YQYJ26INXUBND34H4NRFUZUM',
 'name': 'projects/earthengine-legacy/operations/YQYJ26INXUBND34H4NRFUZUM',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1648081763550}

In [ ]:
totPixExport.status()

In [ ]:
#@title Display the classification as mean over imagecollection
visParams3 = {
  'bands': ['ClassificationOnCollection'],
  'min': 0,
  'max': 1,
  'gamma': 0.4,
  'opacity': 0.6
}

visParams4 = {
  'bands': ['TotalPixels'],
  'min': 0,
  'max': 50,
  'gamma': 0.4,
  'opacity': 0.6
}

visParams5 = {
  'bands': ['Accuracy'],
  'min': 0,
  'max': 1,
  'gamma': 0.4,
  'opacity': 0.6
}

# Create a folium map object.
Panama_map2 = folium.Map(location=[8.384, -78.9165], zoom_start=7.5, height=1000)


Panama_map2.add_ee_layer(totalPixels,visParams4,'Available Pixels (unclouded)')
Panama_map2.add_ee_layer(LCmean,visParams3,'Average classification across all images in the collection')
Panama_map2.add_ee_layer(PanamaLC, visParams2, 'Original 2012 Data')

# Add a layer control panel to the map.
Panama_map2.add_child(folium.LayerControl())
display(Panama_map2)

In [ ]:
# @title Download the Median composite if we want to see band distribution by classification
compositeFile = 'L8_Me_2019_20'

compositeExport = ee.batch.Export.image.toDrive(image=composite.select(bands).unmask(-9999).toFloat(),
                                     region=roi.getInfo()['coordinates'],
                                     description='my_description',
                                     folder='GEE_exports',
                                     fileNamePrefix=compositeFile,
                                     scale=analysisScale,
                                     maxPixels=3e9)
compositeExport.start()
compositeExport.status()

{'creation_timestamp_ms': 1648400882464,
 'description': 'my_description',
 'id': 'ZDZXPM4TDI4524IFWC7JEMQZ',
 'name': 'projects/earthengine-legacy/operations/ZDZXPM4TDI4524IFWC7JEMQZ',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1648400882464}

In [ ]:
compositeExport.status()

{'creation_timestamp_ms': 1594498609536,
 'description': 'my_description',
 'destination_uris': ['https://drive.google.com/#folders/11g_8fgEvOR3u3gBYDeXaJWOpRQGIz8R7',
  'https://drive.google.com/#folders/11g_8fgEvOR3u3gBYDeXaJWOpRQGIz8R7',
  'https://drive.google.com/#folders/11g_8fgEvOR3u3gBYDeXaJWOpRQGIz8R7',
  'https://drive.google.com/#folders/11g_8fgEvOR3u3gBYDeXaJWOpRQGIz8R7',
  'https://drive.google.com/#folders/11g_8fgEvOR3u3gBYDeXaJWOpRQGIz8R7',
  'https://drive.google.com/#folders/11g_8fgEvOR3u3gBYDeXaJWOpRQGIz8R7'],
 'id': 'HMFTF5K7AHEUEA725OASNLXO',
 'name': 'projects/earthengine-legacy/operations/HMFTF5K7AHEUEA725OASNLXO',
 'start_timestamp_ms': 1594501275643,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1594502432803}

In [ ]:
!geeadd tasks

Tasks Running: 1
Tasks Pending: 3
Tasks Completed: 14
Tasks Failed: 2
Tasks Cancelled: 2


In [ ]:
trainingexport = ee.batch.Export.image.toAsset(
                          image = composite_training,
                          description = 'Landsat8_training_2019w',
                          assetId = 'users/reetam1/Landsat8_training_2019w',
                          scale = 20,
                          maxPixels = 3e9,
                          region = roi,
                          pyramidingPolicy = {
                            '.default': 'sample'
                          }
                )
trainingexport.start()